# Accuracy Test

## 📦 **Fractional Derivative Package Comparison**

### **Test Function:**

* **Function:** `sin(x)`
* **Order α:** 0.5
* **Domain:** \[0.0, 1.57]
* **Points:** 1024

---

### **Absolute Error Comparison**

| Method          | differintP\_err | differint\_err | differintC\_err |
| :-------------- | --------------: | -------------: | --------------: |
| GL              |          0.0871 |         0.0871 |          0.0871 |
| GLpoint         |          0.0871 |         0.0868 |          0.0871 |
| GLI             |          0.0853 |         0.0853 |               — |
| GLpoint\_direct |          0.0868 |              — |               — |
| RL              |          0.0873 |         0.0873 |          0.0873 |
| RLpoint         |          0.0873 |         0.0873 |          0.0873 |
| CaputoL1point   |          0.0873 |         0.0873 |               — |

*Note:* `—` indicates not available in that package.

---

### **Package Versions**

| Package    | Version |
| :--------- | :------ |
| differintP | 0.0.3   |
| differint  | 1.0.0   |
| differintC | 0.0.2.4 |

In [57]:
import numpy as np
from differintP import *
from differintP.special import GLpoint_direct
import pandas as pd

import differintP as dp

# Legacy (pip install differint)
import differint.differint as d

# C-accelerated (if you have it built/installed)
# pip install differintC
import differintC as dc

## Helper Functions

In [52]:
def accuracy_point(
    gt, # ground truth
    function,
    alpha,
    test_subject,
    start: float = 0.0,
    end: float = 1.0,
    num_points: int = 1024,
):
    res = test_subject(alpha, function, start, end, num_points)
    abs_e = abs(gt - res)
    r2_e = (gt - res) ** 2

    return {
        "name" : test_subject.__name__,
        "abs_e" : abs_e,
        "r2_e" : r2_e,
        "gt" : gt,
        "res": res,
    }

def accuracy_point_from_array(
    gt, # ground truth
    function,
    alpha,
    test_subject,
    start: float = 0.0,
    end: float = 1.0,
    num_points: int = 1024,
):
    res = test_subject(alpha, function, start, end, num_points)[-1]
    abs_e = abs(gt - res)
    r2_e = (gt - res) ** 2

    return {
        "name" : test_subject.__name__,
        "abs_e" : abs_e,
        "r2_e" : r2_e,
        "gt" : gt,
        "res": res,
    }

def accuracy_array(
    gt, # ground truth function
    function,
    alpha,
    test_subject,
    start: float = 0.0,
    end: float = 1.0,
    num_points: int = 1024,
):
    x = np.linspace(start, end, num_points)
    gt_val = gt(x)
    res = test_subject(alpha, function, start, end, num_points)[-1]

    abs_e = abs(gt - gt_val)
    avg_abs_e = np.mean(abs_e)
    max_abs_e = np.max(avg_abs_e)

    return {
        "name" : test_subject.__name__,
        "avg_abs_e" : avg_abs_e,
        "max_abs_e" : max_abs_e,
        "gt_val" : gt_val,
        "res": res,
    }


## Comparison helpers

def try_array(gt, f, alpha, func, *args, **kwargs):
    if func is None:
        return None
    try:
        res = func(alpha, f, *args, **kwargs)
        val = res[-1]
        abs_e = abs(gt - val)
        return val, abs_e
    except Exception as e:
        return str(e), None

def try_point(gt, f, alpha, func, *args, **kwargs):
    if func is None:
        return None
    try:
        val = func(alpha, f, *args, **kwargs)
        abs_e = abs(gt - val)
        return val, abs_e
    except Exception as e:
        return str(e), None
    

## Sin Test

In [53]:
# 1. Setup: Parameters and Ground Truth

# Parameters for the test
alpha = 0.5
start = 0.0
end = np.pi / 2  # Let's pick [0, pi/2] for better nonzero behavior
num_points = 1024


# Test function
f_name = "sin"
f = np.sin
gt_expr = f"sin({end:.2f} + {alpha} * pi / 2)"
gt = np.sin(end + alpha * np.pi / 2)



print("Ground truth (analytical):", gt_expr, "=", gt)

Ground truth (analytical): sin(1.57 + 0.5 * pi / 2) = 0.7071067811865476


In [65]:
# 2.A Run All Methods on This Test

methods_point = [GLpoint, GLpoint_direct, RLpoint, CaputoL1point]
methods_array = [GL, GLI, RL]

results = []

# Test point functions
for method in methods_point:
    result = accuracy_point(
        gt=gt,
        function=f,
        alpha=alpha,
        test_subject=method,
        start=start,
        end=end,
        num_points=num_points,
    )
    results.append(result)

# Test array functions (use last value)
for method in methods_array:
    result = accuracy_point_from_array(
        gt=gt,
        function=f,
        alpha=alpha,
        test_subject=method,
        start=start,
        end=end,
        num_points=num_points,
    )
    results.append(result)
df = pd.DataFrame(results)[["name", "abs_e", "r2_e", "gt", "res"]]

In [66]:
# 3.B Display Results as a Table
print("Setup:")
print(f"a: {alpha} - start: {start} - end: {end:.2f} - points: {num_points} - test: {f_name}")

df
# print(df.to_markdown(index=False))

Setup:
a: 0.5 - start: 0.0 - end: 1.57 - points: 1024 - test: sin


,name,abs_e,r2_e,gt,res
0,GLpoint,0.087064,0.007580,0.707107,0.620043
1,GLpoint_direct,0.086761,0.007528,0.707107,0.620346
2,RLpoint,0.087301,0.007621,0.707107,0.619806
3,CaputoL1point,0.087301,0.007621,0.707107,0.619806
4,GL,0.087064,0.007580,0.707107,0.620043
5,GLI,0.085314,0.007278,0.707107,0.621793
6,RL,0.087301,0.007621,0.707107,0.619806


### Package Comparison

In [80]:
# 2.B Intra Package Test
test_subjects = [
    ('GL',            dp.GL,            d.GL,            dc.GL),
    ('GLpoint',       dp.GLpoint,       d.GLpoint,       dc.GLpoint),
    ('GLI',           dp.GLI,           d.GLI,           None),
    ('GLpoint_direct',GLpoint_direct,   None,            None),
    ('RL',            dp.RL,            d.RL,            dc.RL),
    ('RLpoint',       dp.RLpoint,       d.RLpoint,       dc.RLpoint),
    ('CaputoL1point', dp.CaputoL1point, d.CaputoL1point, None),
]

table = []
for name, f_p, f_l, f_c in test_subjects:
    row = {'Method': name}
    # Choose point or array dispatcher
    dispatcher = try_point if 'point' in name else try_array
    for tag, fn in zip(['differintP', 'differint', 'differintC'], [f_p, f_l, f_c]):
        result = dispatcher(gt, f, alpha, fn, start, end, num_points)
        if result is None:
            row[tag] = ''
        else:
            val, abs_e = result
            row[tag] = f"{val:.8g}" if abs_e is not None else val
            row[tag + "_err"] = f"{abs_e:.2e}" if abs_e is not None else ''
    table.append(row)
df_intra = pd.DataFrame(table)

err_cols = [col for col in df_intra.columns if col.strip().endswith('_err')]
if 'Method' in df_intra.columns:
    err_cols = ['Method'] + err_cols
df_errors = df_intra[err_cols]

In [ ]:
# 3.B Display Results as a Table
print("Setup:")
print(f"a: {alpha} - start: {start} - end: {end:.2f} - points: {num_points} - test: {f_name}")

df_errors
# print(df_errors.to_markdown(index=False))


Setup:
a: 0.5 - start: 0.0 - end: 1.57 - points: 1024 - test: sin
| Method         |   differintP_err |   differint_err |   differintC_err |
|:---------------|-----------------:|----------------:|-----------------:|
| GL             |           0.0871 |          0.0871 |           0.0871 |
| GLpoint        |           0.0871 |          0.0868 |           0.0871 |
| GLI            |           0.0853 |          0.0853 |         nan      |
| GLpoint_direct |           0.0868 |        nan      |         nan      |
| RL             |           0.0873 |          0.0873 |           0.0873 |
| RLpoint        |           0.0873 |          0.0873 |           0.0873 |
| CaputoL1point  |           0.0873 |          0.0873 |         nan      |


## Package Comparison:

| Method         |   differintP_err |   differint_err |   differintC_err |
|:---------------|-----------------:|----------------:|-----------------:|
| GL             |           0.0871 |          0.0871 |           0.0871 |
| GLpoint        |           0.0871 |        + 0.0868 |           0.0871 |
| GLI            |           0.0853 |          0.0853 |         nan      |
| GLpoint_direct |           0.0868 |        nan      |         nan      |
| RL             |           0.0873 |          0.0873 |           0.0873 |
| RLpoint        |           0.0873 |          0.0873 |           0.0873 |
| CaputoL1point  |           0.0873 |          0.0873 |         nan      |

- Setup:
- a: 0.5 - start: 0.0 - end: 1.57 - points: 1024 - test: sin

* Versions

| Package        |   differintP     |   differint     |   differintC_err |
|:---------------|-----------------:|----------------:|-----------------:|
| Version        |           0.0.3  |          1.0.0  |         0.0.2.4  |

## differintP result

| name           |     abs_e |       r2_e |       gt |      res |
|:---------------|----------:|-----------:|---------:|---------:|
| GLpoint        | 0.0870642 | 0.00758018 | 0.707107 | 0.620043 |
| GLpoint_direct | 0.0867613 | 0.00752752 | 0.707107 | 0.620346 |
| RLpoint        | 0.0873008 | 0.00762142 | 0.707107 | 0.619806 |
| CaputoL1point  | 0.0873008 | 0.00762142 | 0.707107 | 0.619806 |
| GL             | 0.0870642 | 0.00758018 | 0.707107 | 0.620043 |
| GLI            | 0.0853137 | 0.00727842 | 0.707107 | 0.621793 |
| RL             | 0.0873008 | 0.00762142 | 0.707107 | 0.619806 |

v0.0.3 results:
a: 0.5 - start: 0.0 - end: 1.57 - points: 1024 - test: sin